In [ ]:
from google.colab import drive
import os
import pandas as pd
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import torch

import random

from tqdm import tqdm


from numpy import random

from scipy.ndimage.interpolation import rotate

<ipython-input-1-25deaaebb2dd>:16: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import rotate


In [ ]:
config = dict()
config['metadata_dir'] = '/content/drive/My Drive/Dataset/HAM10000/HAM10000_metadata.csv'
config['image_data_dir'] = '/content/drive/My Drive/Dataset/image_frame'
config['save_data_dir'] = '/content/drive/My Drive/Dataset/save_data/'

In [ ]:
# Acces to data 
drive.mount('/content/drive')

# Load the CSV files for training and validation
metadata = pd.read_csv(config['metadata_dir'])

raw_lesion_name = metadata['dx']

all = dict()
for element in raw_lesion_name:
  if element not in all:
    all[element] = 0
  else:
    all[element] += 1

for k in all:
  all[k] = float((all[k] / 10015) * 100)

print(all)



# the code is totally not use here ............
# Add the .jpg file extension to the filenames in the dataframe
metadata['image_id'] = metadata['image_id'].apply(lambda x: f"{x}.jpg")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'bkl': 10.963554667998004, 'nv': 66.93959061407888, 'df': 1.1382925611582626, 'mel': 11.10334498252621, 'vasc': 1.4078881677483774, 'bcc': 5.122316525212182, 'akiec': 3.2551173240139795}


In [ ]:
class DataProcessor:

  def __init__(self, image_frame_dir, meta_data_dir, random_rotate):

    # Dir of Image Frame and Metadata:
    self.image_frame_dir = image_frame_dir
    self.meta_data_dir = meta_data_dir

    # Training Image Data: NT * H * W * C    
    self.training_img_data = None  # a list of image idx
    self.train_diag_result = []

    # Training Image Data: NV * H * W * C  
    self.validation_img_data = None  # a list of image idx
    self.valid_diag_result = []

    # geometry feature
    self.rotate = random_rotate
    self._temp_imgID_leision_finder = self._label_image_matcher()
    


  def _label_transformer(self):
    """
    Transform lesion name (bkl ...) to a integer.
    """
    train_metadata = pd.read_csv(self.meta_data_dir)
    raw_lesion_name = train_metadata['dx']

    processed_name = dict()

    count = 0
    for lesion in raw_lesion_name:
      if lesion not in processed_name:
        processed_name[lesion] = count
        count += 1

    return processed_name


  def _label_image_matcher(self):
    """
    Match each image with its label.

    {'img_011': 0, 'img_012': 2, .....}
    """
    train_metadata = pd.read_csv(self.meta_data_dir)
    image_ids = list(train_metadata['image_id'])
    lesion_ids = list(train_metadata['dx'])

    assert len(image_ids) == len(lesion_ids)

    finder = dict()
    self.lebal_info = self._label_transformer()

    for i in range(len(image_ids)):
      if image_ids[i] in finder:
        raise Exception('Duplicate Image Exists !')
      finder[image_ids[i]] = self.lebal_info[lesion_ids[i]]

    return finder


  
  def _split_train_valid(self, train_portion):
    """
    Split the data into: 
      - train_portion% of training
      - (1-train_portion) of validation
    """
    assert train_portion < 1

    temp_imgID_leision_finder = self._label_image_matcher()
    
    full_info = {0: [], 1: [], 2:[], 3: [], 4: [], 5: [], 6: []}

    train_image_idx = []
    valid_image_idx = []

    for image_id in temp_imgID_leision_finder:

      lesion = temp_imgID_leision_finder[image_id]
      
      full_info[lesion].append(image_id)


    for lesion in full_info:

      lesion_images = full_info[lesion]

      full_len = len(lesion_images)
      train_len = int(np.floor(train_portion * full_len))

      train_image_idx += lesion_images[0:train_len + 1]
      valid_image_idx += lesion_images[train_len + 1:]

      assert len(lesion_images[0:train_len + 1]) + \
      len(lesion_images[train_len + 1:]) == full_len

    random.shuffle(train_image_idx)
    random.shuffle(valid_image_idx)

    return train_image_idx, valid_image_idx



  def _read_image(self, train_portion):
  
    train_img_idx, valid_image_idx = self._split_train_valid(train_portion)

    curr_dir = self.image_frame_dir
    
    print('..... Processing Training Image and Label .....')
    for img_dir in tqdm(train_img_idx):

      full_curr_dir = curr_dir + '/' + img_dir + '.jpg'

      image = cv.imread(full_curr_dir)
      image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

      # generate random angle between 0 and 359
      random_angle = random.randint(0, 360)
      rotated_image = rotate(image, random_angle)

      resized_rotate = np.expand_dims(cv.resize(rotated_image, dsize=(299, 299)), axis=0)
      resized_original = np.expand_dims(cv.resize(image, dsize=(299, 299)), axis=0)

      img_id = img_dir.split('.')[0]
      temp_result = self._temp_imgID_leision_finder[img_id]
      self.train_diag_result.append(temp_result)
      self.train_diag_result.append(temp_result)

      if self.training_img_data is None:
          self.training_img_data = resized_original
          self.training_img_data = np.concatenate((self.training_img_data, resized_rotate), axis=0)
      else:
        self.training_img_data = \
        np.concatenate((self.training_img_data, resized_original), axis=0)
        self.training_img_data = np.concatenate((self.training_img_data, resized_rotate), axis=0)


    print('..... Processing Validation Image and Label .....')
    for img_dir in tqdm(valid_image_idx):

      full_curr_dir = curr_dir + '/' + img_dir + '.jpg'
      
      image = cv.imread(full_curr_dir)
      image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

      # random_angle = random.randint(0, 360)
      # rotated_image = rotate(image, random_angle)


      resized_rotate = np.expand_dims(cv.resize(rotated_image, dsize=(299, 299)), axis=0)
      # resized_original = np.expand_dims(cv.resize(image, dsize=(299, 299)), axis=0)

      img_id = img_dir.split('.')[0]
      temp_result = self._temp_imgID_leision_finder[img_id]
      self.valid_diag_result.append(temp_result)
      # self.valid_diag_result.append(temp_result)

      if self.validation_img_data is None:
          self.validation_img_data = resized_original
        #   self.validation_img_data = \
        # np.concatenate((self.validation_img_data, resized_rotate), axis=0)

      else:
        self.validation_img_data = \
        np.concatenate((self.validation_img_data, resized_original), axis=0)
        # self.validation_img_data = \
        # np.concatenate((self.validation_img_data, resized_rotate), axis=0)

    print('.....Image Processing is Completed......')


  def start_processing(self):
    """
    The Final Function to run ....
    """

    self._read_image(0.7)

    print('== Finished Image Reading ==')

    assert len(self.training_img_data) == len(self.train_diag_result)
    assert len(self.validation_img_data) == len(self.valid_diag_result)

    print('Start saving data ....')
    train_data = self.training_img_data
    print('Finish Extract Data')
    train_label = np.array(self.train_diag_result)
    print('Finish Extract Label')

    shuffler = np.random.permutation(len(train_data))
    train_data = train_data[shuffler]
    train_label = train_label[shuffler]
    print('Finish Shuffle Data')


    print('Start saving data ....')
    valid_data = self.validation_img_data
    print('Finish Extract Data')
    valid_label = np.array(self.valid_diag_result)
    print('Finish Extract Label')

    shuffler = np.random.permutation(len(valid_data))
    valid_data = valid_data[shuffler]
    valid_label = valid_label[shuffler]
    print('Finish Shuffle Data')
    
    print('Connecting to Drive ....')
    #drive.mount('/content/drive')
    print('Connected to Drive ....')

    with open(config['save_data_dir'] + 'augfull_train_data.npy', 'wb') as td:
      print('starting saving training ....')
      with tqdm(total=train_data.nbytes, unit='B', unit_scale=True, desc=config['save_data_dir'] + 'augfull_train_data.npy') as pbar:
        np.save(td, train_data)
        pbar.update(train_data.nbytes)

    with open(config['save_data_dir'] + 'augfull_train_label.npy', 'wb') as tl:
      with tqdm(total=train_label.nbytes, unit='B', unit_scale=True, desc=config['save_data_dir'] + 'augfull_train_label.npy') as pbar:
        np.save(tl, train_label)
        pbar.update(train_label.nbytes)

    with open(config['save_data_dir'] + 'augfull_valid_data.npy', 'wb') as vd:
      with tqdm(total=valid_data.nbytes, unit='B', unit_scale=True, desc=config['save_data_dir'] + 'augfull_valid_data.npy') as pbar:
        np.save(vd, valid_data)
        pbar.update(valid_data.nbytes)

    with open(config['save_data_dir'] + 'augfull_valid_label.npy', 'wb') as vl:
      with tqdm(total=valid_label.nbytes, unit='B', unit_scale=True, desc=config['save_data_dir'] + 'augfull_valid_label.npy') as pbar:
        np.save(vl, valid_label)
        pbar.update(valid_label.nbytes)
        #np.save(vl, valid_label)

In [ ]:
dts = DataProcessor(config['image_data_dir'], config['metadata_dir'], False)
dts.start_processing()

..... Processing Training Image and Label .....


100%|██████████| 7014/7014 [4:52:04<00:00,  2.50s/it]


..... Processing Validation Image and Label .....


100%|██████████| 3001/3001 [48:32<00:00,  1.03it/s]


.....Image Processing is Completed......
== Finished Image Reading ==
Start saving data ....
Finish Extract Data
Finish Extract Label
Finish Shuffle Data
Start saving data ....
Finish Extract Data
Finish Extract Label
Finish Shuffle Data
Connecting to Drive ....
Connected to Drive ....
starting saving training ....


/content/drive/My Drive/Dataset/save_data/augfull_train_data.npy: 100%|██████████| 3.76G/3.76G [00:15<00:00, 239MB/s]
/content/drive/My Drive/Dataset/save_data/augfull_train_label.npy: 100%|██████████| 112k/112k [00:00<00:00, 40.4MB/s]
/content/drive/My Drive/Dataset/save_data/augfull_valid_data.npy: 100%|██████████| 805M/805M [00:02<00:00, 306MB/s]
/content/drive/My Drive/Dataset/save_data/augfull_valid_label.npy: 100%|██████████| 24.0k/24.0k [00:00<00:00, 7.79MB/s]


In [ ]:
def proportion_checker(training_label_dir, validation_label_dir):
    drive.mount('/content/drive')


    l0, l1, l2, l3, l4, l5, l6 = 0, 0, 0, 0, 0, 0, 0

    train_label = np.load(training_label_dir).tolist()
    t = len(train_label)

    for label in train_label:
      if label == 0:
        l0 += 1
      if label == 1:
        l1 += 1
      if label == 2:
        l2 += 1
      if label == 3:
        l3 += 1
      if label == 4:
        l4 += 1
      if label == 5:
        l5 += 1
      if label == 6:
        l6 += 1


    print('Training Set: Label0 {}%, Label1: {}%, Label2: {}%, Label3: {}%, Label4: {}%, Label5: {}, Label6: {}'
    .format(l0*100/t, l1*100/t, l2*100/t, l3*100/t, l4*100/t, l5*100/t, l6*100/t))



    l0, l1, l2, l3, l4, l5, l6 = 0, 0, 0, 0, 0, 0, 0

    valid_label = np.load(validation_label_dir).tolist()
    t = len(valid_label)

    for label in valid_label:
      if label == 0:
        l0 += 1
      if label == 1:
        l1 += 1
      if label == 2:
        l2 += 1
      if label == 3:
        l3 += 1
      if label == 4:
        l4 += 1
      if label == 5:
        l5 += 1
      if label == 6:
        l6 += 1

    
    print('Validation Set: Label0 {}%, Label1: {}%, Label2: {}%, Label3: {}%, Label4: {}%, Label5: {}, Label6: {}'
    .format(l0*100/t, l1*100/t, l2*100/t, l3*100/t, l4*100/t, l5*100/t, l6*100/t))

In [ ]:
training_label_dir = config['save_data_dir'] + 'aug1000_train_label.npy'
validation_label_dir = config['save_data_dir'] + 'aug1000_valid_label.npy'

proportion_checker(training_label_dir, validation_label_dir)